In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
keras = tf.keras
layers = tf.keras.layers

In [4]:
train_image_path = glob.glob('./dataset/dc_2000/train/*/*.jpg')

In [5]:
train_image_label = [int(p.split('\\')[1] == 'cat') for p in train_image_path]

In [6]:
def load_preprosess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[256,256])
    image = tf.cast(image,tf.float32)
    image = image/255
    return image,label

In [7]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label))

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [9]:
train_image_ds = train_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)

In [13]:
train_image_ds = train_image_ds.shuffle(train_count).repeat().batch(BATCH_SIZE)

In [11]:
train_image_ds

<ParallelMapDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>

In [12]:
BATCH_SIZE = 32
train_count = len(train_image_path)

In [14]:
test_image_path = glob.glob('./dataset/dc_2000/test/*/*.jpg')
test_image_label = [int(p.split('\\')[1] == 'cat') for p in test_image_path]
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path,test_image_label))
test_image_ds = test_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)
test_image_ds = test_image_ds.repeat().batch(BATCH_SIZE)

In [15]:
test_count = len(test_image_path)
test_count

1000

keras内置经典网络实现

In [16]:
covn_base = keras.applications.VGG16(weights='imagenet',include_top=False)
# weights='imagenet'使用在ImageNet上预训练好的网络，若等于null表示直接使用VGG网络
# include_top是否包含最后的输出层（预训练好的分类器）

In [17]:
covn_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [18]:
model = keras.Sequential()
model.add(covn_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 14,977,857
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 保持不动conv_base中的权重,将其设置为不可训练
covn_base.trainable = False

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 263,169
Non-trainable params: 14,714,688
_________________________________________________________________


In [22]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.0005),
             loss='binary_crossentropy',
             metrics=['acc'])

In [23]:
history = model.fit(
    train_image_ds,
    epochs=4,
    validation_data=test_image_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    validation_steps=test_count//BATCH_SIZE)

Epoch 1/4
62/62 [==============================] - 1231s 20s/step - loss: 0.4629 - acc: 0.7888 - val_loss: 0.3068 - val_acc: 0.8891
Epoch 2/4
62/62 [==============================] - 1194s 19s/step - loss: 0.2926 - acc: 0.8805 - val_loss: 0.2466 - val_acc: 0.9042
Epoch 3/4
62/62 [==============================] - 1179s 19s/step - loss: 0.2389 - acc: 0.9068 - val_loss: 0.3127 - val_acc: 0.8538
Epoch 4/4
62/62 [==============================] - 1178s 19s/step - loss: 0.2278 - acc: 0.9068 - val_loss: 0.2979 - val_acc: 0.8720


In [24]:
# 解冻卷积层
covn_base.trainable = True

In [25]:
len(covn_base.layers)

19

In [26]:
# 调整最后三层
fine_tune_at = -3

In [27]:
# 除了最后三层将其他所有层设置为不可训练
for layer in covn_base.layers[:fine_tune_at]:
    layer.trainable = False

In [29]:
# 将学习速率调小，下探极值
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(lr=0.0005/10),
             metrics=['accuracy'])

In [30]:
# 初始训练epoch
initial_epochs = 4
# 微调后训练epoch
fine_tune_epochs = 10
total_epochs = initial_epochs+fine_tune_epochs

history = model.fit(
    train_image_ds,
    epochs=total_epochs,
    initial_epoch=initial_epochs,
    validation_data=test_image_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    validation_steps=test_count//BATCH_SIZE
)

Epoch 5/14
62/62 [==============================] - 1265s 20s/step - loss: 0.1942 - accuracy: 0.9254 - val_loss: 0.1455 - val_accuracy: 0.9375
Epoch 6/14
62/62 [==============================] - 1200s 19s/step - loss: 0.0928 - accuracy: 0.9693 - val_loss: 0.1574 - val_accuracy: 0.9365
Epoch 7/14
62/62 [==============================] - 1193s 19s/step - loss: 0.0519 - accuracy: 0.9849 - val_loss: 0.1220 - val_accuracy: 0.9546
Epoch 8/14
62/62 [==============================] - 1190s 19s/step - loss: 0.0265 - accuracy: 0.9960 - val_loss: 0.1323 - val_accuracy: 0.9516
Epoch 9/14
62/62 [==============================] - 1194s 19s/step - loss: 0.0173 - accuracy: 0.9975 - val_loss: 0.1292 - val_accuracy: 0.9476
Epoch 10/14
62/62 [==============================] - 1190s 19s/step - loss: 0.0124 - accuracy: 0.9975 - val_loss: 0.1168 - val_accuracy: 0.9556
Epoch 11/14
62/62 [==============================] - 1190s 19s/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.1379 - val_accuracy: 0.94